In [24]:
from docx import *
from docx.shared import RGBColor
import pandas as pd
import re

In [25]:
def toStr(s):  
    str = " " 
    return (str.join(s))

In [26]:
def clean(x):
    # hilangkan digit 
    temp1 = re.sub(r"\d","", x)

    # ilangkan tab 
    temp2 = re.sub(r"\t","", temp1)

    # hilangkan spasi,tab, newline diakhir dan koma
    result = re.sub(r"\s+$|,|;", "", temp2)
    
    return result

In [27]:
def cleanEnd(x):
    x = re.sub(r"[^a-zA-Z\-\s]", '', x) # menghilangkan karakter selain huruf abjad dan '-'
    x = re.sub(' +', ' ',x).strip() # menghilangkan spasi yang berulang
    x = re.sub("\\s*-\\s*", "-",x).strip() # menghilangkan spasi setelah atau sebelum tanda '-'
    
    return x

In [28]:
def insertData(utama,turunan,gabungan,similiar,kelas,panah,keterangan):
    x = [utama,turunan,gabungan,similiar,kelas,panah,keterangan]
    # print(x)
    y = pd.DataFrame(data=[x],columns=['Kata Dasar','Kata Turunan','Kata Gabungan','Kata Similar','Kelas Kata','Arah Panah','Keterangan'])
    
    return x,y

In [29]:
def ambilKata(word):
    # jika ada tanda () merupakan similiar
    if '(' in word :
        x, y = re.sub(r"[/(].*?[/))=]","",word),re.sub(r"[()=]","",word)
        x, y = re.sub(r"\s$","",x),re.sub(r"\s$","",y)
        utama = x
        similar = y
        tempUtama = utama
                    
    else :
        word = re.sub(r"[()]|\s$|-\s","",word)
        utama = word
        similar = ""
        tempUtama = utama

    return utama,similar,tempUtama

In [30]:
# document = Document("test.docx")
document = Document("file-kbbi-docx/KBBI-A.docx")

korpus = pd.DataFrame(columns=['Kata Dasar','Kata Turunan','Kata Gabungan','Kata Similar','Kelas Kata','Arah Panah','Keterangan'])

kelasKata = ['n','v','a','adv','num','p','pron']
ket = ["Ab","Abr","Ach","Adm","Ag","akr","Anat","Antr","Ar","Arg","ark","Astron","Astr","Astrol","Bg","Bjr","Bio","Biol","Bl","Bld","Bot","Bt","Btk","Brk","cak","Cn","Dag","Dik","Dok","Dn","Dy","Ek","Ekg","El","Ent","Far","Fifol","Fil","Filol","Fils","Fis","Fir","Geo","Geog","Geol","Gra","Graf","Gy","Hd","hid","Hin","Hind","hor","Huk","Hut","Isl","Ikn","Ing","Isi","Isl","Jak","Jb","Jk","Jp","Jw","Jyw","k","Kal","Kap","kas","Kat","kep","Kes","Keu","Kim","kl","kp","Kris","Kom","Komp","lesl","Lay","Ling","Lis","lt","Man","Mat","Md","Mdr","Mek","Mes","Met","Mil","Mk","Mks","Mn","Mu","Mus","ok","Olr","on","Opt","Org","Orl","Pem","Pet","Plb","Pol","Psi","Pr","Prot","Publ","Sas","sb","sd","sel","sj","Sen","Sos","Skr","Skt","Sng","Sos","Stat","Tan","tbl","tas","Tek","terb","Terb","Tern","tld","Tns","Us","zat","Zool"]
kosong = ["", " " , "  "]
tempTurunan =""
tempUtama = ""

for paragraph in document.paragraphs:
    
    utama = ""
    turunan = ""
    gabungan = ""
    similar = ""
    kelas = ""
    panah = ""
    keterangan = ""

    tempKelas = []
    tempKeterangan = []
#     tempUtama = ""
    
    tandaPanah = False
    index = 0
    for run in paragraph.runs:
        
        
        # teks warna merah (sublema)
        if ((run.font.color.rgb == RGBColor(255,0,0))):
#             print(run.text+","+str(index))
            
            # sublema
            if(run.bold):
                
                run.text = clean(run.text)
                
                if run.text not in kosong and tandaPanah != True:
                    
                    # tanda panah di sub lema
                    if (re.search("‹", run.text)) :
#                         print(run.text)
                        if re.search("\d", paragraph.runs[index+2].text):
                            tempPanah = paragraph.runs[index+3].text
#                             panah = tempPanah
                        else :
                            tempPanah = paragraph.runs[index+2].text
#                             panah = tempPanah
                            
                        result = ambilKata(tempPanah)
                        
                        if result[1] != "":
                            panah = result[0]
                            x = insertData(utama,turunan,gabungan,similar,kelas,panah,keterangan)
                            print(x[0])
                            korpus  = korpus.append([x[1]],ignore_index=True)
                        
                            panah = result[1]
                        
                        else :
                            
                            panah = result[0]
                            
                        tandaPanah = True

                    else:

                        # sublema
                        if utama != "" or turunan != "" or kelas != "" :

                            # insert
                            x = insertData(utama,turunan,gabungan,similar,kelas,panah,keterangan)
                            print(x[0])
                            korpus  = korpus.append([x[1]],ignore_index=True)
                            utama = ""

                        # menangani jika -- ga bold
                        if index == 1 and re.search("--", paragraph.runs[index-1].text):

                            run.text = tempUtama+" "+run.text

                            result = ambilKata(run.text)

                            gabungan     = result[0]
                            similar  = result[1]
                            kelas = ""
                            keterangan = ""
                            tempKeterangan = []
                            kelas = ""

                        # menangani jika ~ ga bold
                        elif index == 1 and re.search("~", paragraph.runs[index-1].text):
    #                         print(paragraph.runs[index+1].text)
    #                         print(run.text)
                            run.text = tempTurunan+" "+run.text

                            result = ambilKata(run.text)

                            gabungan     = result[0]
                            similar  = result[1]
                            kelas = ""
                            keterangan = ""

                        else:
                            if re.search("--", run.text):
                                # ganti -- dengan lema (sebelumnya/tempUtama)
                                run.text = re.sub(r"--",tempUtama,run.text)

                                result = ambilKata(run.text)

                                gabungan     = result[0]
                                similar  = result[1]
                                kelas = ""
                                keterangan = ""
                                tempKeterangan = []

                            elif re.search("~", run.text):
    #                             print(run.text)
                                # ganti ~ dengan sublema (sebelumnya/tempTurunan)
                                run.text = re.sub(r"~",tempTurunan,run.text)

                                result = ambilKata(run.text)

                                gabungan     = result[0]
                                similar  = result[1]
                                kelas = ""
                                keterangan = ""
                                tempKeterangan = []

                            else:
                                result = ambilKata(run.text)

                                turunan     = result[0]
                                similar    = result[1]
                                tempTurunan = result[2]
                                kelas = ""
                                keterangan = ""
                                tempKeterangan = []


            elif(run.italic):

                run.text = clean(run.text)
                
                x = run.text.split(" ")

                # keterangan
                for y in x:
                    
                    if (y in kelasKata or y in ket) and y not in tempKeterangan:
                        tempKeterangan.append(y)
                        
                if tempKeterangan != "":
                    for temp in tempKeterangan :
                        if temp in ket:
                            keterangan = " ".join(tempKeterangan)
                
                # kelas kata
                for y in x:
                    
                    if y in kelasKata and y != kelas:
                        if kelas != "":
                            # insert
                            x = insertData(utama,turunan,gabungan,similar,kelas,panah,keterangan)
                            print(x[0])
                            korpus  = korpus.append([x[1]],ignore_index=True)
                        kelas = y  
        
        # teks warna hijau (similar)
        elif ((run.font.color.rgb == RGBColor(0,176,80))):
#             print(run.text)
            if(run.bold):
                
                run.text = clean(run.text)
                
                if run.text not in kosong and re.search("\w", run.text) :
                    
                    similar = re.sub(r"[()=]","",run.text)

#                     print(tempUtama)
                    if re.search("--", similar):
                        # ganti -- dengan lema (sebelumnya/tempUtama)
                        similar = re.sub(r"--",tempUtama,similar)
                
                    elif re.search("~", similar):
                        # ganti -- dengan lema (sebelumnya/tempUtama)
                        similar = re.sub(r"~",tempTurunan,similar)
                
#                     print(similar)

        # teks bukan warna merah (lema)
        else:
            
            # lema
            if(run.bold):
#                 print(run.text)
                run.text = clean(run.text)
                
#                 if run.text not in kosong and re.search("\w", run.text) :
                
                if run.text not in kosong and tandaPanah != True:
                    
                    # tanda panah di lema
                    temp = []
                    if (re.search("‹", run.text)) :
#                         print(run.text)
                        if re.search("\d", paragraph.runs[index+2].text):
                            tempPanah = paragraph.runs[index+3].text
#                             panah = tempPanah
                        else :
                            tempPanah = paragraph.runs[index+2].text
#                             panah = tempPanah
                            
                        result = ambilKata(tempPanah)
                        
                        if result[1] != "":
                            panah = result[0]
                            x = insertData(utama,turunan,gabungan,similar,kelas,panah,keterangan)
                            print(x[0])
                            korpus  = korpus.append([x[1]],ignore_index=True)
                        
                            panah = result[1]
                        
                        else :
                            
                            panah = result[0]
                        
#                         tempUtama = paragraph.runs[index-1].text
#                         utama = tempUtama
                        tandaPanah = True
                        
                    else :
                    
                        # lema
                        if utama != "" or turunan!= "" or kelas != "":
                            # insert
                            x = insertData(utama,turunan,gabungan,similar,kelas,panah,keterangan)
                            print(x[0])
                            korpus  = korpus.append([x[1]],ignore_index=True)

                            turunan = ""

                        result = ambilKata(run.text)

                        utama     = result[0]
                        similar  = result[1]
                        tempUtama = result[2]
                        kelas = ""
                        keterangan = ""
                        tempKeterangan = []
                    
#                     if utama != "" or turunan!= "" or kelas != "":
#                         # insert
#                         x = insertData(utama,turunan,gabungan,similar,kelas,panah,keterangan)
#                         print(x[0])
#                         korpus  = korpus.append([x[1]],ignore_index=True)

#                         turunan = ""

#                     result = ambilKata(run.text)

#                     utama     = result[0]
#                     similar  = result[1]
#                     tempUtama = result[2]
#                     kelas = ""
#                     keterangan = ""
#                     tempKeterangan = []
                
            elif(run.italic):

                run.text = clean(run.text)
                
                x = run.text.split(" ")

                # keterangan
                for y in x:
                    
                    if (y in kelasKata or y in ket) and y not in tempKeterangan:
                        tempKeterangan.append(y)
                        
                if tempKeterangan != "":
                    for temp in tempKeterangan :
                        if temp in ket:
                            keterangan = " ".join(tempKeterangan)
                
                # kelas kata
                for y in x:
                    
                    if y in kelasKata and y != kelas:
                        if kelas != "":
                            # insert
                            x = insertData(utama,turunan,gabungan,similar,kelas,panah,keterangan)
                            print(x[0])
                            korpus  = korpus.append([x[1]],ignore_index=True)
                        kelas = y
            
            # tanda panah yang heading
#             else:
#                 run.text = clean(run.text)
# #                 print(run.text)
                
#                 if run.text not in kosong :
                    
#                     # tanda panah
#                     temp = []
#                     if (re.search("‹", run.text)) :
                        
#                         if re.search("\d", paragraph.runs[index+2].text):
#                             tempPanah = paragraph.runs[index+3].text
#                             panah = tempPanah
#                         else :
#                             tempPanah = paragraph.runs[index+2].text
#                             panah = tempPanah
                                     
#                         tempUtama = paragraph.runs[index-1].text
#                         utama = tempUtama
                    
        index += 1
    
    # insert
    if utama not in kosong or turunan not in kosong or gabungan not in kosong or similar not in kosong or kelas not in kosong or panah not in kosong or keterangan not in kosong :
        x = insertData(cleanEnd(utama),cleanEnd(turunan),cleanEnd(gabungan),cleanEnd(similar),cleanEnd(kelas),cleanEnd(panah),cleanEnd(keterangan))
        print(x[0])
        korpus  = korpus.append([x[1]],ignore_index=True)
    

['A', '', '', 'a', 'n', '', '']
['A', '', '', '', 'n', '', '']
['a', '', '', '', 'n', '', '']
['ab', '', '', '', 'n', '', 'ark n']
['aba', '', '', '', 'n', '', '']
['aba-aba', '', '', '', 'n', '', '']
['abad', '', '', '', 'n', '', '']
['', '', 'abad kekal', '', '', '', '']
['', '', 'abad keemasan', '', '', '', '']
['', '', 'abad pertengahan', '', '', '', '']
['', '', 'berabad-abad', '', 'v', '', '']
['', '', 'seabad', '', 'n', '', '']
['abadi', '', '', '', 'a', '', '']
['', '', 'mengabadi', '', 'v', '', '']
['', '', 'mengabadikan', '', 'v', '', '']
['', '', 'pengabadian', '', 'n', '', '']
['', '', 'keabadian', '', 'n', '', '']
['abadiah', '', '', '', 'n', '', 'Ar n']
['abah', '', '', '', 'n', '', '']
['', '', 'mengabah', '', 'v', '', '']
['', '', 'mengabahkan', '', 'v', '', '']
['abah-abah', '', '', '', 'n', '', '']
['', '', 'abah-abah kuda', '', '', '', '']
['', '', 'abah-abah perahu', '', '', '', '']
['', '', 'abah-abah tenun', '', '', '', '']
['abai', '', '', '', 'a', '', '']
['', '

['acara', '', '', '', '', '', '']
['', '', 'mengacara', '', 'v', '', '']
['acaram', '', '', '', 'n', '', 'kl n']
['', '', '', '', '', '', 'Dag']
['acat', '', '', '', 'n', '', '']
['', '', 'acatan', '', 'n', '', '']
['acau', '', '', '', 'v', '', '']
['', '', 'mengacau', '', 'v', '', '']
['acawi', '', '', '', 'n', '', 'ark n']
['acerang', '', '', '', 'n', '', 'n Bot']
['aci', '', '', '', 'a', '', '']
['', '', 'mengacikan', '', 'v', '', '']
['aci', '', '', '', 'n', '', '']
['', '', 'aci sagu', '', '', '', '']
['aci-aci', '', '', '', 'p', '', '']
['', '', 'aci-aci nya', '', '', '', '']
['', '', 'mengaci-ngacikan', '', 'v', '', '']
['', '', 'aci-acian', '', 'n', '', '']
['aci-aci', '', '', '', 'n', '', 'n Lay']
['', '', 'aci-aci kalimat', '', '', '', '']
['acik', '', '', '', 'n', '', 'ark n']
['aco', '', '', '', '', '', '']
['', '', 'mengaco', '', 'v', '', '']
['', '', 'mengaco belo', '', '', '', '']
['aco-acoan', '', '', '', 'a', '', '']
['acu', '', '', '', 'v', '', '']
['', '', 'mengacu',

['', '', 'aduk-adukan', '', 'v', '', '']
['', '', 'pengaduk', '', 'n', '', '']
['', '', 'aduk-adukan', '', 'v', '', '']
['', '', 'adukan', '', 'n', '', '']
['adun', '', '', '', 'a', '', 'kl a']
['', '', 'adukan temadun', '', '', '', '']
['', '', 'beradun', '', 'v', '', '']
['', '', 'mengadunkan', '', 'v', '', '']
['', '', 'adunan', '', 'n', '', '']
['', '', 'pengadun', '', 'n', '', '']
['adven', '', '', '', 'n', '', 'n Kris']
['adverbia', '', '', '', 'n', '', 'n Ling']
['advertensi', '', '', '', 'n', '', '']
['advis', '', '', '', 'n', '', '']
['advokat', '', '', '', 'n', '', '']
['aedes', '', '', '', 'n', '', '']
['aerial', '', '', '', 'n', '', '']
['aerob', '', '', '', 'n', '', 'n Biol']
['aerob', '', '', '', 'n', '', '']
['aerobatik', '', '', '', 'n', '', '']
['aerobik', '', '', '', 'a', '', 'a Biol']
['aerobika', '', '', '', 'n', '', '']
['aerobioskop', '', '', '', 'n', '', 'n Biol']
['aerodinamika', '', '', '', 'n', '', '']
['', '', 'aerodinamika supersonik', '', 'n', '', '']
['aer

['ahlulbait', '', '', '', 'n', '', '']
['ahlulkitab', '', '', '', 'n', '', '']
['ahlulkubur', '', '', '', 'n', '', '']
['ahlunnujum', '', '', '', 'n', '', '']
['ahlusuluk', '', '', '', 'n', '', '']
['ahlusunah', '', '', '', 'n', '', '']
['ahmak', '', '', '', 'a', '', 'kl a']
['ahmar', '', '', '', 'a', '', '']
['aho', '', '', '', 'p', '', '']
['ahsan', '', '', '', 'a', '', '']
['ai', '', '', '', 'p', '', '']
['aib', '', '', '', 'a', '', '']
['', '', 'mengaibkan', '', 'v', '', '']
['', '', 'keaiban', '', 'v', '', '']
['ain', '', '', '', 'n', '', '']
['ainulbanat', '', '', '', 'n', '', 'kl n']
['ainunjariah', '', '', '', 'n', '', '']
['air', '', '', '', 'n', '', '']
['', '', 'air abu', '', '', '', '']
['', '', 'air alas', '', '', '', '']
['', '', 'air ampuh', '', '', '', '']
['', '', 'air anggur', '', '', '', '']
['', '', 'air anjing', '', '', '', '']
['', '', 'air api', '', '', '', '']
['', '', 'air artesis', '', '', '', '']
['', '', 'air arwah', '', '', '', '']
['', '', 'air atar', '', 

['', '', 'akar samping', '', '', '', 'Bot']
['', '', 'akar serabut', '', '', '', 'Bot']
['', '', 'akar susu', '', '', '', 'Bot']
['', '', 'akar teriba', '', '', '', 'Bot']
['', '', 'akar tikus', '', '', '', 'Bot']
['', '', 'akar tinggal', '', '', '', 'Bot']
['', '', 'akar tombak', '', '', '', 'Bot']
['', '', 'akar tuba', '', '', '', 'Bot']
['', '', 'akar tunggang', '', '', '', 'Bot']
['', '', 'akar tunjang', '', '', '', 'Bot']
['', '', 'akar udara', '', '', '', 'Bot']
['', '', 'akar umbi', '', '', '', 'Bot']
['', '', 'akar wangi', '', '', '', 'Bot']
['', '', 'berakar', '', 'v', '', '']
['', '', 'perakaran', '', 'n', '', '']
['akarologi', '', '', '', 'n', '', '']
['akas', '', '', '', 'a', '', 'ark a']
['', '', 'keakasan', '', 'n', '', '']
['akasia', '', '', '', 'n', '', '']
['', '', 'akasia kuning', '', '', '', '']
['akatalepsia', '', '', '', 'a', '', 'a Psi']
['akbar', '', '', '', 'a', '', 'Ar a']
['akhir', '', '', '', 'n', '', '']
['', '', 'akhir minggu', '', '', '', '']
['', '', 'akh

['', '', 'kealaman', '', 'n', '', '']
['alam', '', '', '', 'v', '', '']
['', '', 'mengalami', '', 'v', '', '']
['', '', 'berpengalaman', '', 'v', '', '']
['alam', '', '', '', 'n', '', 'kl n']
['alamanda', '', '', '', 'n', '', '']
['alamas', '', '', '', 'n', '', 'ark n']
['alamat', '', '', '', 'n', '', '']
['', '', '', '', '', '', 'kl']
['', '', '', '', '', '', 'ark']
['', '', 'beralamat', '', 'v', '', '']
['', '', 'mengalamati', '', 'v', '', '']
['alamatulhayat', '', '', '', 'n', '', '']
['alami', '', '', '', 'a', '', '']
['alamiah', '', '', '', 'a', '', '']
['alamin', '', '', '', 'n', '', 'ark n']
['alan-alan', '', '', '', 'n', '', 'kl n']
['alang', '', '', '', 'a', '', '']
['', '', 'beralang', '', 'v', '', '']
['', '', 'mengalang', '', 'v', '', '']
['', '', 'mengalangi', 'mengalang-alangi', 'v', '', '']
['', '', 'mengalangkan', '', 'v', '', '']
['', '', 'teralang', '', 'v', '', '']
['', '', 'alangan', '', 'n', '', '']
['', '', 'beralangan', '', 'v', '', '']
['', '', 'pengalang', '', 

['alir', '', '', '', 'n', '', '']
['', '', 'mengalir', '', 'v', '', '']
['alir', '', '', '', 'a', '', '']
['alis', '', '', '', 'n', '', '']
['alit', '', '', '', 'n', '', '']
['', '', 'mengalit', '', 'v', '', '']
['alit', '', '', '', 'n', '', '']
['', '', 'mengalit', '', 'v', '', '']
['', '', 'mengalit bisul', '', '', '', '']
['', '', 'mengalit gasing', '', '', '', '']
['', '', 'mengalit perut', '', '', '', '']
['alit', '', '', '', 'a', '', '']
['aliterasi', '', '', '', 'n', '', '']
['aliyah', '', '', '', 'n', '', 'n Dik']
['aljabar', '', '', '', 'n', '', '']
['Aljalil', '', '', '', 'n', '', '']
['Alkabir', '', '', '', 'n', '', '']
['Alkadir', '', '', '', 'n', '', '']
['alkah', '', '', '', 'n', '', '']
['', '', '', '', '', '', 'kl']
['alkali', '', '', '', 'n', '', 'n Kim']
['', '', 'kealkalian', '', 'n', '', '']
['alkalinitas', '', '', '', 'n', '', '']
['alkaloid', '', '', '', 'n', '', 'n Kim']
['alkalometri', '', '', '', 'n', '', 'n Kim']
['alkamar', '', '', '', 'n', '', 'kl n']
['alka

['', '', 'bagian berat', '', '', '', '']
['', '', 'bagian contoh', '', '', '', '']
['', '', 'bagian dihati', 'bagian dikehati', '', '', '']
['', '', 'bagian gambar', '', '', '', '']
['', '', 'bagian haluan', '', '', '', '']
['', '', 'bagian hati', '', '', '', '']
['', '', 'bagian ibarat', 'bagian ibarat contoh teladan', '', '', '']
['', '', 'bagian ingatan', '', '', '', '']
['', '', 'bagian inisiatif', '', '', '', '']
['', '', 'bagian jalan', '', '', '', '']
['', '', 'bagian keputusan', '', '', '', '']
['', '', 'bagian ke-sempatan', 'bagian ke-sempatan peluang', '', '', '']
['', '', 'bagian langkah', '', '', '', '']
['', '', 'bagian layar', '', '', '', '']
['', '', 'bagian menantu', '', '', '', '']
['', '', 'bagian muka', '', '', '', '']
['', '', 'bagian panjang', '', '', '', '']
['', '', 'bagian pedoman', '', '', '', '']
['', '', 'bagian peduli', '', '', '', '']
['', '', 'bagian ringkas', '', '', '', '']
['', '', 'bagian resiko', '', '', '', '']
['', '', 'bagian sempena', '', '', '', 

['ana-', '', '', '', 'n', '', '']
['anabolisme', '', '', '', 'n', '', 'n Bio']
['', '', '', '', '', '', 'Kim']
['anadontia', '', '', '', 'n', '', '']
['anaerob', '', '', '', 'n', '', '']
['anafase', '', '', '', 'n', '', '']
['anafilaksis', '', '', '', 'n', '', 'n Far']
['anafora', '', '', '', 'n', '', 'n Ling']
['anaforesis', '', '', '', 'n', '', '']
['anafrodisiak', '', '', '', 'n', '', 'n Far']
['anai-anai', '', '', '', 'n', '', '']
['anaIgesik', '', '', '', 'n', '', 'n Far']
['anak', '', '', '', 'n', '', '']
['', '', 'anak adam', '', '', '', '']
['', '', 'anak air', '', '', '', '']
['', '', 'anak ajaib', '', '', '', 'Psi']
['', '', 'anak alang', '', '', '', '']
['', '', 'ampang', '', '', '', '']
['', '', 'anak angin', '', '', '', '']
['', '', 'anak angkat', '', '', '', '']
['', '', 'anak baju', '', '', '', '']


KeyboardInterrupt: 

In [ ]:
korpus.to_excel (r'Done/Korpus-Z.xlsx', index = False, header=True)